In [ ]:
import sys; sys.path.extend([snakemake.params.scripts])

from propensity_matching import propensity_score_matching
from ukb_data import load

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import pandas as pd
from scipy.stats import bootstrap
import numpy as np
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
df, t1_feature_names, t1_feature_fids, treatment_col, match_cols, tmask_col, mask_col =  load(snakemake)

In [ ]:
n_patients = df[df[treatment_col]].shape[0]
print(n_patients)
if n_patients > 5000:
    print(f"Too many patients ({n_patients}) for propensity matching, dropping {n_patients-5000} patients")
    to_drop = n_patients - 5000
    df.loc[df[df[treatment_col]].sample(to_drop).index, treatment_col] = pd.NA
df_subset = df.dropna(subset=treatment_col)
df_subset[treatment_col] = df_subset[treatment_col].astype(bool)

matching, stats = propensity_score_matching(df_subset, treatment_col, match_cols, mask_col)

eids_cn = matching[matching[treatment_col]==False].index.to_list()
eids_dx = matching[matching[treatment_col]==True].index.to_list()

exclude_eids = matching.index.to_list()
if tmask_col is not None:
    exclude_eids += df.loc[df[tmask_col]].index.to_list()

x = df.loc[~df.index.isin(exclude_eids)][t1_feature_fids].values
y = df.loc[~df.index.isin(exclude_eids)]['age_t2'].values
x_cn = df.loc[eids_cn][t1_feature_fids].values
x_dx = df.loc[eids_dx][t1_feature_fids].values
y_cn = df.loc[eids_cn]['age_t2'].values
y_dx = df.loc[eids_dx]['age_t2'].values

stats

In [ ]:
scaler = StandardScaler()

x_scaled = scaler.fit_transform(x)
x_cn_scaled = scaler.transform(x_cn)
x_dx_scaled = scaler.transform(x_dx)

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, train_size=int(snakemake.wildcards.ntrain),test_size=1000, random_state=int(snakemake.wildcards.seed))

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
df_res = pd.DataFrame()
df_res['mask'] = snakemake.wildcards.mask
df_res['tmask'] = snakemake.wildcards.trainmask
df_res['treatment'] = snakemake.wildcards.icd_code
df_res['matching'] = snakemake.wildcards.matching
df_res['ntrain'] = int(snakemake.wildcards.ntrain)
df_res['ndx'] = len(x_dx)


In [ ]:
for alpha in np.logspace(-2, 10, 25):
    ridge_model = Ridge(alpha=alpha)
    ridge_model.fit(x_train, y_train)
    y_train_pred = ridge_model.predict(x_train)
    y_test_pred = ridge_model.predict(x_test)
    bag_train = y_train - y_train_pred

    df_res.loc[alpha,'r2_train'] = r2_score(y_train, y_train_pred)
    df_res.loc[alpha,'mae_train'] = mean_absolute_error(y_train, y_train_pred)
    df_res.loc[alpha,'r2_test'] = r2_score(y_test, y_test_pred)
    df_res.loc[alpha,'mae_test'] = mean_absolute_error(y_test, y_test_pred)

    y_cn_pred = ridge_model.predict(x_cn_scaled)
    y_dx_pred = ridge_model.predict(x_dx_scaled)

    bag_cn = y_cn - y_cn_pred
    bag_dx = y_dx - y_dx_pred

    bag_cn_corr = bag_cn - LinearRegression().fit(y_train.reshape(-1, 1),bag_train).predict(y_cn.reshape(-1, 1))
    bag_dx_corr = bag_dx - LinearRegression().fit(y_train.reshape(-1, 1),bag_train).predict(y_dx.reshape(-1, 1))

    func = lambda a,b: (np.mean(a)-np.mean(b))/np.sqrt((np.std(a)**2+np.std(b)**2)/2)

    effect = func(bag_cn,bag_dx)
    sem = bootstrap((bag_cn, bag_dx), func,paired=True).standard_error

    effect_corr = func(bag_cn_corr,bag_dx_corr)
    sem_corr = bootstrap((bag_cn_corr, bag_dx_corr), func,paired=True).standard_error

    df_res.loc[alpha,'effect'] = effect
    df_res.loc[alpha,'sem'] = sem
    df_res.loc[alpha,'effect_corr'] = effect_corr
    df_res.loc[alpha,'sem_corr'] = sem_corr
    
    print(df_res.loc[alpha])

df_res.to_json(snakemake.output.effects, orient='table')


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def do_plot(df_res, title):
    # Create two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

    # Plot R^2 on the primary y-axis with blue color in the first subplot
    sns.lineplot(data=df_res, x=df_res.index, y='r2_test', label='R^2', ax=ax1, color='blue')
    ax1.set_ylabel('R^2')
    ax1.set_title('R^2, Effect Size vs Alpha - {}'.format(title))

    # Create a secondary y-axis
    ax1b = ax1.twinx()
    # Plot MAE on the secondary y-axis with red color
    sns.lineplot(data=df_res, x=df_res.index, y='mae_test', label='MAE', ax=ax1b, color='red')
    ax1b.set_ylabel('MAE')


    # Plot the effect size with error bars from the sem column in the second subplot
    sns.lineplot(data=df_res, x=df_res.index, y='effect', label='Effect size', color='green', ax=ax2)
    ax2.errorbar(df_res.index, df_res['effect'], yerr=df_res['sem'], fmt='o', color='green')
    ax2.set_xlabel('Alpha')
    ax2.set_ylabel('Effect size')

    ax1.set_xscale('log')
    ax2.set_xscale('log')

    # Add vertical lines through the maximal value of the upper plot and the minimal value of the lower plot
    ax1.axvline(df_res['r2_test'].idxmax(), color='blue', linestyle='--')
    ax2.axvline(df_res['r2_test'].idxmax(), color='blue', linestyle='--')
    ax1.axvline(df_res['effect'].idxmax(), color='green', linestyle='--')
    ax2.axvline(df_res['effect'].idxmax(), color='green', linestyle='--')

    ax1.axhline(df_res['r2_test'].max(), color='blue', linestyle='--')
    ax1.axhline(df_res['r2_test'].loc[df_res['effect'].idxmax()], color='green', linestyle='--')
    ax2.axhline(df_res['effect'].max(), color='green', linestyle='--')
    ax2.axhline(df_res['effect'].loc[df_res['r2_test'].idxmax()], color='blue', linestyle='--')
    
    # Create a shared legend for ax1 and ax2
    fig.legend(loc='upper right')

    # Adjust the layout to prevent overlapping of plots
    fig.tight_layout()

    # Show the plot
    plt.show()

In [ ]:
do_plot(df_res, snakemake.wildcards.icd_code)
